In [1]:
#import required libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
from difflib import SequenceMatcher


In [2]:
#request html content, both for regular season and playoffs
url_base_reg = 'https://www.basketball-reference.com/leagues/NBA_{}.html'
url_base_play = 'https://www.basketball-reference.com/playoffs/NBA_{}.html'
url_base_payroll = 'https://hoopshype.com/salaries/{}/'

years = [year for year in range(2013, 2023)]

teams = ["Boston Celtics", "Brooklyn Nets", "New York Knicks", "Philadelphia 76ers", "Toronto Raptors", "Chicago Bulls",
"Cleveland Cavaliers", "Detroit Pistons", "Indiana Pacers", "Milwaukee Bucks", "Atlanta Hawks", "Charlotte Hornets",
"Miami Heat", "Orlando Magic", "Washington Wizards", "Denver Nuggets", "Minnesota Timberwolves", "Oklahoma City Thunder",
"Portland Trail Blazers", "Utah Jazz", "Golden State Warriors", "Los Angeles Clippers", "Los Angeles Lakers", "Phoenix Suns",
"Sacramento Kings", "Dallas Mavericks", "Houston Rockets", "Memphis Grizzlies", "New Orleans Pelicans", "San Antonio Spurs", "Charlotte Bobcats", "New Orleans Hornets"]

df_main = pd.DataFrame()

cpi = pd.read_csv('data/cpi-u_2013-22.csv')
bri = pd.read_csv('data/total_league_revenues.csv')

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [3]:
for i in years:

    url_reg = url_base_reg.format(i)

    per_game = requests.get(url_reg)

    #create BeautifulSoup object
    soup = BeautifulSoup(per_game.content, 'html.parser')

    #locate correct table
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="per_game-team") 
    rows = table.findAll(lambda tag: tag.name=='tr')

    #create DataFrame
    df = pd.read_html(str(table))[0]

    #clean DataFrame
    df = df.drop(index=30)
    df['Team'] = df['Team'].str.replace('*', '')
    df.insert(2, "Year", i, True)
    df = df.drop(columns=['Rk', 'G', 'MP'])


    #add advanced stats to DataFrame
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="advanced-team") 
    rows = table.findAll(lambda tag: tag.name=='tr')
    df_adv = pd.read_html(str(table))[0]
    
    df_adv.columns = df_adv.columns.droplevel()
    df_adv = df_adv.drop(index=30)
    df_adv['Team'] = df_adv['Team'].str.replace('*', '')
    df_adv = df_adv.drop(columns=["Rk", "L", "PW", "PL", "Unnamed: 17_level_1", "Unnamed: 22_level_1", "Unnamed: 27_level_1", "Arena"])


    #add playoff wins to DataFrame
    url_play = url_base_play.format(i)
    playoff_wins = requests.get(url_play)
    soup = BeautifulSoup(playoff_wins.content, 'html.parser')
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="advanced-team")
    df_play = pd.read_html(str(table))[0]
    
    df_play.columns = df_play.columns.droplevel()
    df_play = df_play.drop(index=16)
    df_play = df_play.rename(columns={"Tm": "Team", "W" : "Playoff_W"})
    df_play = df_play[["Team", "Playoff_W"]]


    #add payroll data
    url_payroll = url_base_payroll.format(f'{i-1}-{i}')
    payrolls = requests.get(url_payroll)
    soup = BeautifulSoup(payrolls.content, 'html.parser')
    table = soup.find("table", class_="hh-salaries-ranking-table hh-salaries-table-sortable responsive")
    rows = table.findAll(lambda tag: tag.name=='tr')
    df_payroll = pd.read_html(str(table))[0]

    df_payroll = df_payroll.iloc[:, [1,2]]
    df_payroll[f'{i-1}/{i-2000}'] = df_payroll[f'{i-1}/{i-2000}'].str.replace('[$,]', '').astype(int)
    df_payroll = df_payroll.rename(columns={f'{i-1}/{i-2000}': 'Payroll'})

    team_replace = {row: team for row in df_payroll['Team'].to_list() for team in teams if similar(row, team)>=0.53}

    for index, team in team_replace.items():
        df_payroll.loc[df_payroll['Team'] == index, 'Team'] = team
            
    # adjust payroll for inflation, basketball-related-income
    df_payroll['Adjusted Payroll'] = df_payroll['Payroll'] / (cpi[cpi['Year'] == (i - 1)]['Annual'].values[0] 
    + bri[bri['Year'] == (i - 1)]['Revenue'].values[0])


    # merge all datasets
    df = pd.merge(df, df_adv, on='Team')
    df = pd.merge(df, df_play, on='Team')
    df = pd.merge(df, df_payroll, on='Team')

    df_main = pd.concat([df_main, df])
    df_main = df_main.reset_index(drop=True)


C:\Users\whisk\AppData\Local\Temp\ipykernel_9392\3324215985.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Team'] = df['Team'].str.replace('*', '')
C:\Users\whisk\AppData\Local\Temp\ipykernel_9392\3324215985.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_adv['Team'] = df_adv['Team'].str.replace('*', '')
C:\Users\whisk\AppData\Local\Temp\ipykernel_9392\3324215985.py:57: FutureWarning: The default value of regex will change from True to False in a future version.
  df_payroll[f'{i-1}/{i-2000}'] = df_payroll[f'{i-1}/{i-2000}'].str.replace('[$,]', '').astype(int)
C:\Users\whisk\AppData\Local\Temp\ipykernel_9392\3324215985.py:19: FutureWarning: The default value

In [5]:
df_main

,Team,Year,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,...,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Attend.,Attend./G,Playoff_W,Payroll,Adjusted Payroll
0,Minnesota Timberwolves,2022,41.6,91.0,0.457,14.8,41.3,0.358,26.8,49.7,...,0.198,0.535,14.2,74.9,0.227,657148.0,16028.0,2.0,137432702,150698.709387
1,Memphis Grizzlies,2022,43.5,94.4,0.461,11.5,32.7,0.353,32.0,61.7,...,0.180,0.523,13.3,77.8,0.195,646785.0,15775.0,6.0,117284457,128605.608737
2,Milwaukee Bucks,2022,41.8,89.4,0.468,14.1,38.4,0.366,27.8,51.0,...,0.199,0.536,11.6,78.6,0.165,715581.0,17453.0,7.0,160875421,176404.290711
3,Phoenix Suns,2022,43.7,90.1,0.485,11.6,31.9,0.364,32.1,58.2,...,0.176,0.510,13.0,77.1,0.195,663171.0,16175.0,7.0,136476474,149650.179282
4,Atlanta Hawks,2022,41.5,88.3,0.470,12.9,34.4,0.374,28.6,53.9,...,0.205,0.543,11.5,76.9,0.177,672742.0,16408.0,1.0,135166020,148213.230698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Brooklyn Nets,2013,35.9,79.8,0.450,7.7,21.5,0.357,28.2,58.3,...,0.219,0.503,13.1,73.7,0.174,704702.0,17188.0,3.0,87644649,146662.531752
151,Boston Celtics,2013,37.0,79.7,0.465,6.1,17.2,0.358,30.9,62.6,...,0.204,0.482,14.2,73.3,0.216,744960.0,18624.0,2.0,75508399,126354.011252
152,Indiana Pacers,2013,35.1,80.6,0.436,6.9,19.7,0.347,28.3,60.8,...,0.219,0.453,12.9,74.6,0.197,626069.0,15270.0,11.0,66107195,110622.253570
153,Memphis Grizzlies,2013,36.1,81.5,0.444,4.7,13.5,0.345,31.5,68.0,...,0.202,0.475,15.2,74.3,0.209,681613.0,16625.0,8.0,63054115,105513.300000


In [7]:
#write data to csv
df_main.to_csv('data/playoff_per_game_2013-22.csv')